In [163]:
import click
import pandas as pd
import numpy as np
import sys
import json
pd.set_option('display.max_colwidth', 600)
sys.path.append('../src')

from data_loader import InputDataLoader
from feature_creators import UserFeaturizer

In [2]:
input_data = InputDataLoader('../data', '../data/input')

In [3]:
input_data.train_df[input_data.train_df.type == 'Assessment'].tail(10)

Loading train data ...
Loaded train data for 7.43 seconds.


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
11337816,91561152,5448d652309a6324,2019-09-22 02:07:22.816000+00:00,"{""bucket"":3,""buckets_placed"":[3,1,2],""target_bucket"":1,""coordinates"":{""x"":269,""y"":371,""stage_width"":1015,""stage_height"":762},""correct"":false,""prompt"":""holds least"",""event_count"":52,""game_time"":623...",ffeb0b1b,52,4025,62346,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337817,2dcad279,5448d652309a6324,2019-09-22 02:07:22.818000+00:00,"{""description"":""Tap the bucket that holds the least amount of liquid."",""identifier"":""Dot_TapBucketLeast"",""media_type"":""audio"",""duration"":2319,""event_count"":53,""game_time"":62346,""event_code"":3110}",ffeb0b1b,53,3110,62346,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337818,923afab1,5448d652309a6324,2019-09-22 02:07:22.821000+00:00,"{""description"":""Look for the smallest bucket."",""identifier"":""Dot_SmallestBucket"",""media_type"":""audio"",""total_duration"":1210,""event_count"":54,""game_time"":62346,""event_code"":3010}",ffeb0b1b,54,3010,62346,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337819,2dcad279,5448d652309a6324,2019-09-22 02:07:24.061000+00:00,"{""description"":""Look for the smallest bucket."",""identifier"":""Dot_SmallestBucket"",""media_type"":""audio"",""duration"":1266,""event_count"":55,""game_time"":63612,""event_code"":3110}",ffeb0b1b,55,3110,63612,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337820,3ee399c3,5448d652309a6324,2019-09-22 02:07:25.737000+00:00,"{""coordinates"":{""x"":482,""y"":365,""stage_width"":1015,""stage_height"":762},""event_count"":56,""game_time"":65281,""event_code"":4070}",ffeb0b1b,56,4070,65281,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337821,28520915,5448d652309a6324,2019-09-22 02:07:27.562000+00:00,"{""misses"":1,""prompt"":""holds least"",""mode"":""selecting"",""round_number"":2,""duration"":7067,""event_count"":58,""game_time"":67094,""event_code"":2030}",ffeb0b1b,58,2030,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337822,91561152,5448d652309a6324,2019-09-22 02:07:27.562000+00:00,"{""bucket"":1,""buckets_placed"":[3,1,2],""target_bucket"":1,""coordinates"":{""x"":552,""y"":401,""stage_width"":1015,""stage_height"":762},""correct"":true,""prompt"":""holds least"",""event_count"":57,""game_time"":6709...",ffeb0b1b,57,4025,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337823,d3268efa,5448d652309a6324,2019-09-22 02:07:27.566000+00:00,"{""description"":""Awesome."",""identifier"":""Dot_Awesome"",""media_type"":""audio"",""total_duration"":700,""event_count"":59,""game_time"":67094,""event_code"":3021}",ffeb0b1b,59,3021,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337824,b5053438,5448d652309a6324,2019-09-22 02:07:28.311000+00:00,"{""description"":""Awesome."",""identifier"":""Dot_Awesome"",""media_type"":""audio"",""duration"":753,""event_count"":60,""game_time"":67847,""event_code"":3121}",ffeb0b1b,60,3121,67847,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337825,2b058fe3,5448d652309a6324,2019-09-22 02:07:31.963000+00:00,"{""session_duration"":71511,""event_count"":61,""game_time"":71511,""event_code"":2010}",ffeb0b1b,61,2010,71511,Cauldron Filler (Assessment),Assessment,MAGMAPEAK


In [4]:
input_data.train_df[(input_data.train_df.type == 'Assessment') & (input_data.train_df.event_code == 2010)].groupby(['installation_id', 'title']).count()


event_id  game_session  \
installation_id title                                                  
0006a69f        Bird Measurer (Assessment)           1             1   
                Mushroom Sorter (Assessment)         3             3   
00129856        Bird Measurer (Assessment)           1             1   
001d0ed0        Cart Balancer (Assessment)           1             1   
                Mushroom Sorter (Assessment)         2             2   
...                                                ...           ...   
ffc90c32        Cart Balancer (Assessment)           1             1   
                Chest Sorter (Assessment)            1             1   
                Mushroom Sorter (Assessment)         2             2   
ffd2871d        Cauldron Filler (Assessment)         1             1   
ffeb0b1b        Cauldron Filler (Assessment)         2             2   

                                              timestamp  event_data  \
installation_id title                                                 
0006a69f        Bird Measurer (Assessment)            1           1   
                Mushroom Sorter (Assessment)          3           3   
00129856        Bird Measurer (Assessment)            1           1   
001d0ed0        Cart Balancer (Assessment)            1           1   
                Mushroom Sorter (Assessment)          2           2   
...                                                 ...         ...   
ffc90c32        Cart Balancer (Assessment)            1           1   
                Chest Sorter (Assessment)             1           1   
                Mushroom Sorter (Assessment)          2           2   
ffd2871d        Cauldron Filler (Assessment)          1           1   
ffeb0b1b        Cauldron Filler (Assessment)          2           2   

                                              event_count  event_code  \
installation_id title                                                   
0006a69f        Bird Measurer (Assessment)              1           1   
                Mushroom Sorter (Assessment)            3           3   
00129856        Bird Measurer (Assessment)              1           1   
001d0ed0        Cart Balancer (Assessment)              1           1   
                Mushroom Sorter (Assessment)            2           2   
...                                                   ...         ...   
ffc90c32        Cart Balancer (Assessment)              1           1   
                Chest Sorter (Assessment)               1           1   
                Mushroom Sorter (Assessment)            2           2   
ffd2871d        Cauldron Filler (Assessment)            1           1   
ffeb0b1b        Cauldron Filler (Assessment)            2           2   

                                              game_time  type  world  
installation_id title                                                 
0006a69f        Bird Measurer (Assessment)            1     1      1  
                Mushroom Sorter (Assessment)          3     3      3  
00129856        Bird Measurer (Assessment)            1     1      1  
001d0ed0        Cart Balancer (Assessment)            1     1      1  
                Mushroom Sorter (Assessment)          2     2      2  
...                                                 ...   ...    ...  
ffc90c32        Cart Balancer (Assessment)            1     1      1  
                Chest Sorter (Assessment)             1     1      1  
                Mushroom Sorter (Assessment)          2     2      2  
ffd2871d        Cauldron Filler (Assessment)          1     1      1  
ffeb0b1b        Cauldron Filler (Assessment)          2     2      2  

[6639 rows x 9 columns]

In [5]:
input_data.train_df[(input_data.train_df.game_session == '5448d652309a6324') ]


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
11337765,90d848e0,5448d652309a6324,2019-09-22 02:06:20.595000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0,""event_code"":2000}",ffeb0b1b,1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337766,923afab1,5448d652309a6324,2019-09-22 02:06:20.878000+00:00,"{""description"":""Put the right sized bucket under each statue. Match the bucket to the same sized statue!"",""identifier"":""Dot_PutRightBucket,Dot_MatchBucket"",""media_type"":""audio"",""total_duration"":45...",ffeb0b1b,2,3010,394,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337767,532a2afb,5448d652309a6324,2019-09-22 02:06:20.881000+00:00,"{""buckets"":[2,1,3],""target_bucket"":0,""mode"":""placing"",""prompt"":""N/A"",""round_number"":1,""event_count"":3,""game_time"":394,""event_code"":2020}",ffeb0b1b,3,2020,394,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337768,2dcad279,5448d652309a6324,2019-09-22 02:06:25.539000+00:00,"{""description"":""Put the right sized bucket under each statue. Match the bucket to the same sized statue!"",""identifier"":""Dot_PutRightBucket,Dot_MatchBucket"",""media_type"":""audio"",""duration"":4697,""ev...",ffeb0b1b,4,3110,5091,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337769,3ee399c3,5448d652309a6324,2019-09-22 02:06:26.357000+00:00,"{""coordinates"":{""x"":721,""y"":218,""stage_width"":1015,""stage_height"":762},""event_count"":5,""game_time"":5896,""event_code"":4070}",ffeb0b1b,5,4070,5896,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
...,...,...,...,...,...,...,...,...,...,...,...
11337821,28520915,5448d652309a6324,2019-09-22 02:07:27.562000+00:00,"{""misses"":1,""prompt"":""holds least"",""mode"":""selecting"",""round_number"":2,""duration"":7067,""event_count"":58,""game_time"":67094,""event_code"":2030}",ffeb0b1b,58,2030,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337822,91561152,5448d652309a6324,2019-09-22 02:07:27.562000+00:00,"{""bucket"":1,""buckets_placed"":[3,1,2],""target_bucket"":1,""coordinates"":{""x"":552,""y"":401,""stage_width"":1015,""stage_height"":762},""correct"":true,""prompt"":""holds least"",""event_count"":57,""game_time"":6709...",ffeb0b1b,57,4025,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337823,d3268efa,5448d652309a6324,2019-09-22 02:07:27.566000+00:00,"{""description"":""Awesome."",""identifier"":""Dot_Awesome"",""media_type"":""audio"",""total_duration"":700,""event_count"":59,""game_time"":67094,""event_code"":3021}",ffeb0b1b,59,3021,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
11337824,b5053438,5448d652309a6324,2019-09-22 02:07:28.311000+00:00,"{""description"":""Awesome."",""identifier"":""Dot_Awesome"",""media_type"":""audio"",""duration"":753,""event_count"":60,""game_time"":67847,""event_code"":3121}",ffeb0b1b,60,3121,67847,Cauldron Filler (Assessment),Assessment,MAGMAPEAK


In [29]:
user_sample = input_data.train_df[input_data.train_df.installation_id == '00fc65b6']

In [106]:
user_sample[(user_sample.type == 'Assessment') & (user_sample.event_code == 4020)].loc[53046, 'title']

'Cart Balancer (Assessment)'

In [ ]:
titles_dict = dict()
titles_list = input_data.train_df.title.unique()
for title in titles_list:
    titles_dict[title] = input_data.train_df[input_data.train_df.title == title].iloc[0,:].type

In [266]:
def calculate_assessments_features(user_sample):
    user_features = dict()
    titles = [title for title, type in titles_dict.items() if type in ['Game', 'Assessment']]
    for title in titles:
        user_features[f"{title.lower().replace(' ', '_')}_misses"] = None
        user_features[f"{title.lower().replace(' ', '_')}_duration"] = None

    # both assessments and games have tasks which children should accomplish and they assessed
    assessed_results = user_sample[(user_sample.event_code == 2030) & (user_sample.type.isin(['Game', 'Assessment']))]
    for title, results in assessed_results.groupby('title'):
        misses = [json.loads(event_data)['misses'] for event_data in results.event_data.tolist()]
        print(results.iloc[0, :].type)
        if results.iloc[0, :].type == 'Assessment':
            duration = sum(user_sample[(user_sample.event_code == 2010) & (
                           user_sample.title == title)].game_time.tolist())
            print(duration)
            print(misses)
        else:
#             print(results.loc[results.groupby('game_session').timestamp.idxmax(), :])
            durations = results.loc[results.groupby('game_session').timestamp.idxmax(), 'game_time'].tolist()
            print(durations)
            print(misses)
            duration = sum(durations)

        user_features[f"{title.lower().replace(' ', '_')}_misses"] = sum(misses)
        user_features[f"{title.lower().replace(' ', '_')}_misses_duration"] = sum(misses)/(duration+1)
        user_features[f"{title.lower().replace(' ', '_')}_duration"] = duration
    return user_features

def calculate_basic_stat_features(user_sample):
    features = dict()
    for (title, event_code), results in user_sample.groupby(['title','event_code']):
        print('All:', results.shape)
        features[f"{title}_{event_code}_count"] = results['game_session'].count()
        results = results.loc[results.groupby('game_session').timestamp.idxmax(), :]
        for column in ['event_count', 'game_time']:
            print('last:', results.shape, results[column].to_list(), results[column].max(), results[column].min())
            features[f"{title}_{event_code}_{column}_max"] = results[column].max()
            features[f"{title}_{event_code}_{column}_min"] = results[column].min()
            features[f"{title}_{event_code}_{column}_mean"] = results[column].mean()
    return features

def calculate_placement_accuracy_features(user_sample):
    user_features = dict()
    titles = [title for title, type in titles_dict.items() if type == 'Assessment']
    for title in titles:
        user_features[f"{title.lower().replace(' ', '_')}_placement_acc"] = None
    placement_results = user_sample[( (user_sample.event_code == 4100)) & (user_sample.title.isin(titles))]
    for title, results in placement_results.groupby('title'):
        true_attempts = results['event_data'].str.contains(':true,').sum()
        false_attempts = results['event_data'].str.contains(':false,').sum()
        user_features[f"{title.lower().replace(' ', '_')}_placement_acc"] = true_attempts / (true_attempts + false_attempts) if (true_attempts + false_attempts) != 0 else -1
    return user_features

def calculate_placement_accuracy_feature_current_assessment(title, user_sample):
    user_features = dict()

    user_features["current_assessment_placement_acc"] = None
    placement_results = user_sample[
        ((user_sample.event_code == 4100) | (user_sample.event_code == 4100)) & (user_sample.title == title)]
    true_attempts = placement_results['event_data'].str.contains(':true,').sum()
    false_attempts = placement_results['event_data'].str.contains(':false,').sum()
    print(true_attempts)
    print(false_attempts)
    user_features["current_assessment_placement_acc"] = true_attempts / (
                true_attempts + false_attempts) if (true_attempts + false_attempts) != 0 else -1
    return user_features

In [267]:
calculate_placement_accuracy_features(user_sample)

{'mushroom_sorter_(assessment)_placement_acc': 1.0,
 'bird_measurer_(assessment)_placement_acc': 0.0,
 'cauldron_filler_(assessment)_placement_acc': 0.6666666666666666,
 'cart_balancer_(assessment)_placement_acc': 0.6,
 'chest_sorter_(assessment)_placement_acc': 0.16666666666666666}

In [268]:
calculate_basic_stat_features(user_sample)

All: (1, 11)
last: (1, 11) [1] 1 1
last: (1, 11) [0] 0 0
All: (3, 11)
last: (1, 11) [90] 90 90
last: (1, 11) [155329] 155329 155329
All: (3, 11)
last: (1, 11) [102] 102 102
last: (1, 11) [178230] 178230 178230
All: (1, 11)
last: (1, 11) [4] 4 4
last: (1, 11) [10271] 10271 10271
All: (1, 11)
last: (1, 11) [24] 24 24
last: (1, 11) [28539] 28539 28539
All: (26, 11)
last: (1, 11) [99] 99 99
last: (1, 11) [173197] 173197 173197
All: (2, 11)
last: (1, 11) [86] 86 86
last: (1, 11) [150045] 150045 150045
All: (26, 11)
last: (1, 11) [101] 101 101
last: (1, 11) [175863] 175863 175863
All: (2, 11)
last: (1, 11) [87] 87 87
last: (1, 11) [153846] 153846 153846
All: (1, 11)
last: (1, 11) [2] 2 2
last: (1, 11) [3771] 3771 3771
All: (7, 11)
last: (1, 11) [98] 98 98
last: (1, 11) [173197] 173197 173197
All: (24, 11)
last: (1, 11) [94] 94 94
last: (1, 11) [168796] 168796 168796
All: (3, 11)
last: (1, 11) [100] 100 100
last: (1, 11) [175247] 175247 175247
All: (2, 11)
last: (1, 11) [47] 47 47
last: (1, 1

All: (4, 11)
last: (4, 11) [1, 1, 1, 1] 1 1
last: (4, 11) [0, 0, 0, 0] 0 0
All: (15, 11)
last: (4, 11) [134, 40, 41, 150] 150 40
last: (4, 11) [110153, 69857, 53246, 184043] 184043 53246
All: (15, 11)
last: (4, 11) [149, 65, 68, 167] 167 65
last: (4, 11) [120754, 88825, 73398, 198011] 198011 73398
All: (20, 11)
last: (4, 11) [135, 41, 42, 151] 151 41
last: (4, 11) [110153, 69857, 53246, 184043] 184043 53246
All: (74, 11)
last: (4, 11) [143, 62, 65, 162] 162 62
last: (4, 11) [118520, 86442, 71398, 195694] 195694 71398
All: (20, 11)
last: (4, 11) [153, 69, 72, 171] 171 69
last: (4, 11) [126254, 94343, 78914, 203511] 203511 78914
All: (20, 11)
last: (4, 11) [138, 45, 45, 154] 154 45
last: (4, 11) [115970, 76591, 59563, 190011] 190011 59563
All: (74, 11)
last: (4, 11) [150, 66, 69, 168] 168 66
last: (4, 11) [120754, 88825, 73398, 198011] 198011 73398
All: (20, 11)
last: (4, 11) [154, 70, 73, 172] 172 70
last: (4, 11) [130254, 98343, 82898, 207478] 207478 82898
All: (6, 11)
last: (4, 11) [3

last: (4, 11) [101, 40, 105, 59] 105 40
last: (4, 11) [125748, 63935, 884202, 151276] 884202 63935
All: (21, 11)
last: (4, 11) [99, 51, 98, 74] 99 51
last: (4, 11) [125047, 64631, 878670, 158093] 878670 64631
All: (60, 11)
last: (3, 11) [57, 61, 83] 83 57
last: (3, 11) [68947, 73775, 163479] 163479 68947
All: (4, 11)
last: (4, 11) [1, 1, 1, 1] 1 1
last: (4, 11) [0, 0, 0, 0] 0 0
All: (3, 11)
last: (3, 11) [1, 1, 1] 1 1
last: (3, 11) [0, 0, 0] 0 0
All: (3, 11)
last: (3, 11) [1, 1, 1] 1 1
last: (3, 11) [0, 0, 0] 0 0
All: (9, 11)
last: (3, 11) [40, 103, 29] 103 29
last: (3, 11) [64303, 165783, 104127] 165783 64303
All: (9, 11)
last: (3, 11) [48, 109, 35] 109 35
last: (3, 11) [77020, 188084, 126193] 188084 77020
All: (3, 11)
last: (3, 11) [3, 3, 3] 3 3
last: (3, 11) [10583, 6808, 4720] 10583 4720
All: (2, 11)
last: (2, 11) [18, 6] 18 6
last: (2, 11) [51643, 49573] 51643 49573
All: (1, 11)
last: (1, 11) [6] 6 6
last: (1, 11) [15167] 15167 15167
All: (4, 11)
last: (3, 11) [4, 4, 10] 10 4
last

All: (2, 11)
last: (2, 11) [1, 1] 1 1
last: (2, 11) [0, 0] 0 0


{'Air Show_2000_count': 1,
 'Air Show_2000_event_count_max': 1,
 'Air Show_2000_event_count_min': 1,
 'Air Show_2000_event_count_mean': 1.0,
 'Air Show_2000_game_time_max': 0,
 'Air Show_2000_game_time_min': 0,
 'Air Show_2000_game_time_mean': 0.0,
 'Air Show_2020_count': 3,
 'Air Show_2020_event_count_max': 90,
 'Air Show_2020_event_count_min': 90,
 'Air Show_2020_event_count_mean': 90.0,
 'Air Show_2020_game_time_max': 155329,
 'Air Show_2020_game_time_min': 155329,
 'Air Show_2020_game_time_mean': 155329.0,
 'Air Show_2030_count': 3,
 'Air Show_2030_event_count_max': 102,
 'Air Show_2030_event_count_min': 102,
 'Air Show_2030_event_count_mean': 102.0,
 'Air Show_2030_game_time_max': 178230,
 'Air Show_2030_game_time_min': 178230,
 'Air Show_2030_game_time_mean': 178230.0,
 'Air Show_2060_count': 1,
 'Air Show_2060_event_count_max': 4,
 'Air Show_2060_event_count_min': 4,
 'Air Show_2060_event_count_mean': 4.0,
 'Air Show_2060_game_time_max': 10271,
 'Air Show_2060_game_time_min': 10

In [220]:
calculate_placement_accuracy_feature_current_assessment('Cart Balancer (Assessment)', user_sample)

3
2


{'current_assessment_placement_acc': 0.6}

In [203]:
user_sample[(user_sample.title == 'Bird Measurer (Assessment)') & (user_sample.event_code == 4100)]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
55732,070a5291,469b3ff25bf7824f,2019-09-29 14:52:29.867000+00:00,"{""correct"":false,""hats"":[4,5,8],""event_count"":35,""game_time"":34574,""event_code"":4100}",00fc65b6,35,4100,34574,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [62]:
calculate_assessments_features(user_sample)

Game
[178230]
[0, 0, 0]
Game
[550140, 34563, 111452]
[0, 4, 0, 2, 0, 2, 0]
Assessment
0
[0]
Game
[68613, 275493]
[1, 1, 1, 1, 1]
Assessment
101207
[0, 2, 0]
Assessment
77955
[1, 1, 0, 1]
Assessment
126923
[0, 10]
Game
[120754, 88825, 73398, 198011]
[0, 2, 0, 2, 0, 1, 0, 0, 1, 4, 0, 1, 0, 0, 1]
Game
[158734]
[1, 1, 0, 1, 0, 1, 0, 0, 0]
Game
[259987, 142194]
[0, 0, 9, 0, 1, 0, 0, 1]
Game
[298352, 208015]
[1, 3, 8, 1, 1, 0]
Game
[129664, 886052, 138142]
[0, 0, 2, 1, 2, 2, 0, 2]
Game
[77020, 188084, 126193]
[1, 7, 0, 0, 0, 0, 1, 0, 0]
Assessment
36658
[0]
Game
[299828, 283936, 274264]
[1, 0, 0, 3, 2, 2, 1, 0, 5, 0, 1, 4, 0, 0, 2, 1, 0, 1, 0, 1, 3, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 0, 2, 2, 0, 0, 1, 1, 3, 3, 0, 0]
Game
[428435, 78073, 320866]
[1, 0, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 2, 0, 2, 2, 0, 0, 3, 0, 0, 0, 4, 0, 0, 0, 1, 0, 1, 0, 2, 0, 5, 0, 0, 2, 0, 0, 2, 1, 0, 1, 2, 3, 0, 3, 0, 0, 0, 2, 3, 0, 0, 2, 1, 0, 2, 0, 1, 0]


{'scrub-a-dub_misses': 58,
 'scrub-a-dub_duration': 827374,
 'dino_drink_misses': 14,
 'dino_drink_duration': 506367,
 'all_star_sorting_misses': 8,
 'all_star_sorting_duration': 696155,
 'mushroom_sorter_(assessment)_misses': 0,
 'mushroom_sorter_(assessment)_duration': 36658,
 'air_show_misses': 0,
 'air_show_duration': 178230,
 'crystals_rule_misses': 4,
 'crystals_rule_duration': 158734,
 'bird_measurer_(assessment)_misses': 0,
 'bird_measurer_(assessment)_duration': 0,
 'bubble_bath_misses': 5,
 'bubble_bath_duration': 344106,
 'dino_dive_misses': 11,
 'dino_dive_duration': 402181,
 'chow_time_misses': 12,
 'chow_time_duration': 480988,
 'cauldron_filler_(assessment)_misses': 3,
 'cauldron_filler_(assessment)_duration': 77955,
 'pan_balance_misses': 46,
 'pan_balance_duration': 858028,
 'happy_camel_misses': 9,
 'happy_camel_duration': 1153858,
 'cart_balancer_(assessment)_misses': 2,
 'cart_balancer_(assessment)_duration': 101207,
 'chest_sorter_(assessment)_misses': 10,
 'chest_

In [121]:
codes = set()
for event_data in input_data.train_df[(input_data.train_df.type == 'Activity')].event_data.map(json.loads).to_list():
    if 'correct' in event_data.keys():
        codes.add(event_data['event_code'])
codes

set()

In [199]:
input_data.specs_df.loc['0db6d71d', 'info']

'This event occurs when the player places a chest on one of the three pillars. It contains information about the chest that was placed, the correctness of the action, and where the placement occurred. This event is used to calculate accuracy and to diagnose player strategies and understanding.'